In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [11]:
#store the files for reference
files = !ls | grep turnstile.*txt
working_file = files[0]
census_raw = 'ACS_15_5YR_S1901_with_ann.csv'

In [12]:
df = pd.read_csv(working_file)
df.rename(columns = {
    'C/A': 'area',
    'UNIT': 'unit',
    'SCP': 'scp',
    'STATION': 'station',
    'LINENAME': 'linename',
    'DIVISION': 'division',
    'DATE': 'date',
    'TIME': 'time',
    'DESC': 'desc',
    'ENTRIES': 'entries',
    'EXITS                                                               ': 'exits'},
    inplace = True)
# group = df.groupby(['area','unit', 'scp', 'station', 'date', 'time', 'entries', 'exits'])
# date = pd.DataFrame(group.size().reset_index())
# del date[0]
# date = date.sort_values(by='date').groupby(['station'])


# date['entries'] = date['entries'] - date['entries'].shift()
# date['exits'] = date['exits'] - date['exits'].shift()
# date['daily'] = date['entries']- date['exits']
# date = date[date.time == '20:00:00']


df['hourly'] = df.groupby(['area','unit','scp','station'])['entries'].diff()
df['hourly'].fillna(0, inplace=True)

In [13]:
#set all negative hourly entries to absolute value
df.loc[df['hourly'] < 0, 'hourly'] = abs(df['hourly'])
df[df['hourly'] < 0]

,area,unit,scp,station,linename,division,date,time,desc,entries,exits,hourly


In [14]:
df['hourly'].describe(percentiles=[0.01, 0.999])

count    1.944660e+05
mean     2.534253e+04
std      6.250360e+06
min      0.000000e+00
1%       0.000000e+00
50%      7.500000e+01
99.9%    2.117745e+03
max      1.961790e+09
Name: hourly, dtype: float64

In [15]:
df['good'] = df['hourly'] <= 5000

In [16]:
group = df[df['good']==True].groupby(['station', 'date'])['hourly'].sum()

In [25]:
#import census file and call columns for household income and zip code
census_df = pd.read_csv(census_raw)
census_df = census_df[['GEO.display-label', 'HC01_EST_VC09', 'HC04_EST_VC10','HC01_EST_VC11']]
census_df = census_df.drop(0).reset_index(drop=True)

In [28]:
census_df.rename(columns = { 'GEO.display-label': 'zip',
                           'HC01_EST_VC09': '100_149',
                           'HC04_EST_VC10': '150_199',
                           'HC01_EST_VC11': '200_over'},
                            inplace=True)

In [31]:
census_df['zip'] = census_df['zip'].map(lambda x : str(x)[-5:])
census_df.head()

,zip,100_149,150_199,200_over
0,10001,13.2,10.3,22.1
1,10002,8.5,3.9,5.9
2,10003,15.2,10.9,22.5
3,10004,22.0,12.8,28.0
4,10005,24.6,12.9,27.4
